In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import os
import tarfile
from six.moves import urllib
import pandas as pd
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = ""
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### Data preparation

In [2]:
def dict2list(dict):
    listtmp = []
    for subDict in dict.values():
        tmp=[]
        for item in subDict.keys():
            tmp.append(item)
        listtmp.append(tmp)
    return listtmp

## FP-Growth

In [13]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode  # needs to be updated
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print('  ' * ind, self.name, ':', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

"""
Create FP tree
"""
def createTree(dataSet, minSup=1):  # create FP-tree from dataset but don't mine
    headerTable = {}
    # go over dataSet twice
    for trans in dataSet:  # first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in headerTable.keys():  # remove items not meeting minSup
        if headerTable[k] < minSup:
            del (headerTable[k])
    #print len(headerTable)
    freqItemSet = set(headerTable.keys())
    # print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  # if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]  # reformat headerTable to use Node link
    # print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None)  # create tree
    for tranSet, count in dataSet.items():  # go through dataset 2nd time
        #print "create tree loop"
        localD = {}
        for item in tranSet:  # put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)  # populate tree with ordered freq itemset
    return retTree, headerTable  # return tree and header table

# 让FP树生长
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:  # check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count)  # incrament count
    else:  # add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:  # update header table
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:  # call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 增加节点后更新头指针表
def updateHeader(nodeToTest, targetNode):  # this version does not use recursion
    while (nodeToTest.nodeLink != None):  # Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

"""
Find Conditional pattern base
"""
def ascendTree(leafNode, prefixPath):  # ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)


def findPrefixPath(basePat, treeNode):  # treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats


"""
Create Mine tree
"""
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    #print "mine tree loop"
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1])]  # (sort header table)
    for basePat in bigL:  # start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        # print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        # print 'condPattBases :',basePat, condPattBases
        # 2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        # print 'head from conditional tree: ', myHead
        if myHead != None:  # 3. mine cond. FP-tree
            # print 'conditional tree for: ',newFreqSet
            # myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

# 将数据集转换为字典格式
# frozen set：不可变的set，有哈希值
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [12]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
fpTree, headerTab = createTree(initSet, 3)
freqItems = []
mineTree(fpTree, headerTab, 3, set([]), freqItems)
print(freqItems)

TypeError: '<' not supported between instances of 'treeNode' and 'treeNode'

# Association rules

### Apriori algorithm

In [3]:
'''
Created on Mar 24, 2011
Ch 11 code
@author: Peter
'''
from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                ssCnt.setdefault(can, 0)
                ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        if freqSet-conseq not in supportData or conseq not in supportData: continue
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        kulc = (conf+ supportData[freqSet]/supportData[conseq])/2
        imbalance = abs(supportData[freqSet-conseq]-supportData[conseq])/(supportData[freqSet-conseq]+supportData[conseq]-supportData[freqSet])
        if conf >= minConf: 
            #print(freqSet-conseq,'-->',conseq,'conf:',conf,'kulc:', kulc, 'imbalance:', imbalance)
            brl.append((freqSet-conseq, conseq, conf, kulc, imbalance))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
def pntRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print(itemMeaning[item])
        print("           -------->")
        for item in ruleTup[1]:
            print(itemMeaning[item])
        print("confidence: %f" % ruleTup[2])
        print(' ')      #print a blank line

In [4]:
data = loadDataSet()
L, sup = apriori(data)
rules = generateRules(L, sup)

### Mine association rules

In [5]:
def mine_rules(cluster, train_data, support=0.15, confidence=0.9):
    train_list = dict2list(train_data)
    print('cluster:', cluster)
    print('record numbers:', len(train_data))
    # 频集挖掘
    freq_items, support = apriori(train_list, minSupport=support) # freq_items is a 2D list, support is a dict
    print('frequent items:', len(freq_items))
    ass_rules = generateRules(freq_items, support, confidence) # ass_rules is a list of set (setA, setB, conf)
    print('association rules:', len(ass_rules))
    np.save(path + 'freq_items_' + str(cluster) + '.npy', freq_items)
    np.save(path + 'support_' + str(cluster) + '.npy', support)
    np.save(path + 'ass_rules_' + str(cluster) + '.npy', ass_rules)
    return freq_items, support, ass_rules

In [5]:
path = './birch6_validate/'
train_data = np.load(path + 'user_train_data.npy').item()

In [41]:
# 4 5 cluster duo
cluster = 0
train_list = dict2list(train_data[cluster])
freq_items, support = apriori(train_list, minSupport=0.11)
np.save(path + 'freq_items_' + str(cluster) + '.npy', freq_items)
np.save(path   + 'support_' + str(cluster) + '.npy', support)
print(len(support))

KeyboardInterrupt: 

In [33]:
freq_items

[[frozenset({716682}),
  frozenset({882626}),
  frozenset({716744}),
  frozenset({882654}),
  frozenset({716678}),
  frozenset({716318}),
  frozenset({882537}),
  frozenset({883092}),
  frozenset({716745}),
  frozenset({716640}),
  frozenset({883064}),
  frozenset({883076}),
  frozenset({882617}),
  frozenset({913690}),
  frozenset({716638}),
  frozenset({716251}),
  frozenset({883040}),
  frozenset({883037}),
  frozenset({883036}),
  frozenset({716317}),
  frozenset({716660}),
  frozenset({913473}),
  frozenset({882545}),
  frozenset({716261}),
  frozenset({716440}),
  frozenset({716351}),
  frozenset({913494}),
  frozenset({716238}),
  frozenset({716649}),
  frozenset({716216}),
  frozenset({716271}),
  frozenset({913474}),
  frozenset({716643})],
 [frozenset({882617, 882654}),
  frozenset({716318, 716660}),
  frozenset({716238, 716351}),
  frozenset({716440, 716678}),
  frozenset({716238, 716745}),
  frozenset({716351, 716649}),
  frozenset({716251, 716682}),
  frozenset({882545, 88

In [38]:
results = pd.DataFrame(columns=['cluster', 'n_learners', 'minSup', 'n_freqsets'])
for minSup in range(11,16,1):
    n_row = len(results)
    results.loc[n_row] = [cluster, len(train_list), float(minSup)/100, 0]
    for freqSetList in freq_items:
        for freqSet in freqSetList:
            if freqSet in support and support[freqSet]>float(minSup)/100:
                results.loc[n_row, 'n_freqsets'] += 1
results.to_csv(path+'param_minSup'+str(cluster)+'.csv')

In [39]:
results

,cluster,n_learners,minSup,n_freqsets
0,0.0,8841.0,0.13,889.0
1,0.0,8841.0,0.14,6.0
2,0.0,8841.0,0.15,0.0


In [9]:
results = pd.DataFrame(columns=['cluster', 'n_learners', 'minSup', 'n_freqsets'])
for cluster in range(0, 1):
    train_list = dict2list(train_data[cluster])
    if len(train_list)==0: continue
    freq_items, support = apriori(train_list, minSupport=0.2)
    np.save(path + 'freq_items_' + str(cluster) + '.npy', freq_items)
    np.save(path   + 'support_' + str(cluster) + '.npy', support)
    for minSup in range(20,100,5):
        n_row = len(results)
        results.loc[n_row] = [cluster, len(train_list), float(minSup)/100, 0]
        for freqSetList in freq_items:
            for freqSet in freqSetList:
                if freqSet in support and support[freqSet]>float(minSup)/100:
                    results.loc[n_row, 'n_freqsets'] += 1
results.to_csv(path+'param_minSup.csv')

KeyboardInterrupt: 

In [12]:
i=0
freq_items, support, ass_rules = mine_rules(i, train_data[i], support=0.15, confidence=0.9)
print('done')

cluster: 0
record numbers: 8841


KeyboardInterrupt: 

### Recommend by rules

In [ ]:
# return a dict as CF
def predict(user, train_data, support, ass_rules, k=10):
    predictions = {}
    if user not in train_data: return {}
    train_item_set = train_data[user].keys()
    for rule in ass_rules:
        leftSet = rule[0]
        rightSet = rule[1]
        conf = rule[2]
        if set(leftSet).issubset(train_item_set) and rightSet in support:
            diffSet = set(rightSet) - (set(rightSet) & train_item_set)
            for item in diffSet:
                predictions.setdefault(item, 0.0)
                predictions[item] = max(predictions[item], conf)
    return dict(sorted(predictions.items(), key=lambda e: e[1], reverse=True)[:k])